# Multi-Agent Pipeline 통합 테스트 (Production Level)

## FastAPI Backend 범위 (AI Worker - Multi-Agent Pipeline)

```
Phase 1: Extraction Layer (Level 1)
├── Step 1: Character + Setting (병렬)
└── Step 2: Event (순차)

Phase 2: Analysis Layer (Level 2)
├── Relationship Analyzer
├── Consistency Checker
└── Plot Integrator

Phase 3: Validation Layer (Level 3)
└── ✅ Validator Agent → Spring Boot Callback
```

> **Note**: Validation 완료 후 Spring Boot 서버로 콜백하여 Neo4j 저장 및 Image Generation 처리

In [ ]:
import sys, os, json, asyncio
project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
if project_root not in sys.path: sys.path.insert(0, project_root)
from dotenv import load_dotenv
load_dotenv(os.path.join(project_root, '.env'))
print(f"Project root: {project_root}")

In [ ]:
SAMPLE_STORY = """
어두운 숲 속에서 서진은 검을 쥐고 있었다. 키가 크고 검은 머리카락을 가진 그는 어렸을 때부터 검술을 익혀온 뛰어난 검객이었다.
오래된 거목들이 하늘을 가리고 있었고, 땅에는 짙은 안개가 깔려 있었다. 달빛이 나뭇잎 사이로 희미하게 비추고 있었다.

"우리가 이곳에서 그를 만날 수 있을까?" 서진의 옆에 서 있던 하나가 물었다.
하나는 마을에서 가장 현명한 치료사였고, 항상 차분하고 신중했다. 그녀는 하얀 치료사 로브를 입고 있었다.

갑자기 나무 사이에서 그림자가 나타났다. 그것은 이민호였다. 한때는 서진의 가장 친한 친구였지만,
지금은 그들의 마을을 배신한 적이 되어버렸다. 그의 차가운 눈빛에서 분노와 슬픔이 느껴졌다.

"오랜만이군, 서진." 이민호가 차갑게 말했다. 검은 갑옷을 입은 그는 예전보다 더 날카로워 보였다.
서진은 용감하게 앞으로 나섰다. 긴장감이 최고조에 달했다. "왜 마을을 배신했어, 민호?"

박서연이라는 젊은 여성이 숲에서 뛰어나왔다. 그녀는 마을의 정탐꾼으로, 빠르고 민첩했다.
"경비병들이 오고 있어요! 빨리 결정해야 해요!" 그녀의 목소리에는 급박함이 묻어났다.
"""

def create_base_state(story=SAMPLE_STORY):
    return {"content": story, "project_id": "test-001", "document_id": "doc-001", 
            "job_id": "job-001", "callback_url": "http://localhost:8080/internal/callback",
            "extracted_characters": [], "extracted_settings": [], "extracted_events": [],
            "relationship_graph": {}, "consistency_report": {}, "plot_integration": {},
            "validation_result": {}, "retry_count": 0, 
            "messages": [], "errors": []}

def run_async(coro):
    try:
        loop = asyncio.get_event_loop()
        if loop.is_running():
            import nest_asyncio; nest_asyncio.apply()
            return loop.run_until_complete(coro)
        return asyncio.run(coro)
    except: return asyncio.run(coro)

## Phase 1: Extraction Layer

### Step 1: Master Data Extraction (병렬)
Character + Setting 동시 추출

In [ ]:
from app.agents.extraction.character import character_extraction_node
from app.agents.extraction.setting import setting_extraction_node

async def step1_master_data_extraction(state):
    """Step 1: Character + Setting 병렬 추출"""
    print("\n" + "="*70)
    print("🚀 Phase 1 - Step 1: Master Data Extraction (병렬)")
    print("="*70)
    
    char_result, setting_result = await asyncio.gather(
        character_extraction_node(state),
        setting_extraction_node(state)
    )
    
    state["extracted_characters"] = char_result.get("extracted_characters", [])
    state["extracted_settings"] = setting_result.get("extracted_settings", [])
    
    print(f"\n✅ Characters: {len(state['extracted_characters'])}명")
    print(f"✅ Settings: {len(state['extracted_settings'])}개")
    return state

state = create_base_state()
state = run_async(step1_master_data_extraction(state))

### Step 2: Narrative Flow Extraction (순차)
Event 추출 - Character/Setting 참조

In [ ]:
from app.agents.extraction.event import event_extraction_node

async def step2_event_extraction(state):
    """Step 2: Event 추출"""
    print("\n" + "="*70)
    print("🚀 Phase 1 - Step 2: Narrative Flow Extraction (순차)")
    print("="*70)
    
    event_result = await event_extraction_node(state)
    state["extracted_events"] = event_result.get("extracted_events", [])
    
    print(f"\n✅ Events: {len(state['extracted_events'])}개")
    return state

state = run_async(step2_event_extraction(state))

In [ ]:
# Extraction 결과 확인
print("\n" + "="*70)
print("📊 Phase 1 완료: Extraction Layer 결과")
print("="*70)

print("\n🧑 Characters:")
for char in state["extracted_characters"]:
    print(f"   • {char.get('name')} ({char.get('role')})")

print("\n🏰 Settings:")
for setting in state["extracted_settings"]:
    print(f"   • [{setting.get('setting_id', 'N/A')}] {setting.get('name')}")

print("\n📅 Events:")
for event in state["extracted_events"]:
    print(f"   • [{event.get('event_id')}] {event.get('narrative_summary', event.get('description', 'N/A'))[:40]}...")

## Phase 2: Analysis Layer

Relationship + Consistency + Plot 분석 (병렬)

In [ ]:
from app.agents.analysis.relationship import relationship_analysis_node
from app.agents.analysis.consistency import consistency_check_node

async def phase2_analysis(state):
    """Phase 2: Analysis Layer"""
    print("\n" + "="*70)
    print("🚀 Phase 2: Analysis Layer (병렬)")
    print("="*70)
    
    # 병렬 실행
    rel_result, consist_result = await asyncio.gather(
        relationship_analysis_node(state),
        consistency_check_node(state)
    )
    
    state["relationship_graph"] = rel_result.get("relationship_graph", {})
    state["consistency_report"] = consist_result.get("consistency_report", {})
    
    print(f"\n✅ Relationships: {len(state['relationship_graph'].get('relationships', []))}개")
    conflicts = state['consistency_report'].get('conflicts', [])
    print(f"✅ Conflicts detected: {len(conflicts)}개")
    
    return state

state = run_async(phase2_analysis(state))

In [ ]:
# Analysis 결과 확인
print("\n" + "="*70)
print("📊 Phase 2 완료: Analysis Layer 결과")
print("="*70)

print("\n🔗 Relationships:")
for rel in state["relationship_graph"].get("relationships", []):
    print(f"   • {rel.get('source')} → {rel.get('target')}: {rel.get('type')}")

print("\n🔍 Consistency Report:")
conflicts = state["consistency_report"].get("conflicts", [])
if conflicts:
    for c in conflicts:
        print(f"   ⚠️ [{c.get('severity')}] {c.get('description', c.get('type', 'N/A'))}")
else:
    print("   ✅ 충돌 없음")

## Phase 3: Validation Layer

최종 검증 후 Spring Boot Callback

In [ ]:
from app.agents.validation.validator import validator_node

async def phase3_validation(state):
    """Phase 3: Validation Layer"""
    print("\n" + "="*70)
    print("🚀 Phase 3: Validation Layer")
    print("="*70)
    
    validation_result = await validator_node(state)
    state["validation_result"] = validation_result.get("validation_result", {})
    
    is_valid = state["validation_result"].get("is_valid", False)
    print(f"\n{'✅ Validation PASSED' if is_valid else '❌ Validation FAILED'}")
    
    return state

state = run_async(phase3_validation(state))

## Spring Boot Callback 데이터

Validation 완료 후 Spring Boot 서버로 전송되는 데이터

In [ ]:
callback_data = {
    "job_id": state["job_id"],
    "project_id": state["project_id"],
    "document_id": state["document_id"],
    "status": "completed" if state["validation_result"].get("is_valid", False) else "failed",
    
    # Extraction Results
    "extracted_characters": state["extracted_characters"],
    "extracted_settings": state["extracted_settings"],
    "extracted_events": state["extracted_events"],
    
    # Analysis Results
    "relationship_graph": state["relationship_graph"],
    "consistency_report": state["consistency_report"],
    
    # Validation Result
    "validation_result": state["validation_result"]
}

print("\n" + "="*70)
print("📤 Spring Boot Callback 데이터")
print("="*70)
print(f"POST {state['callback_url']}")
print(f"\nSummary:")
print(f"   • Status: {callback_data['status']}")
print(f"   • Characters: {len(callback_data['extracted_characters'])}개")
print(f"   • Settings: {len(callback_data['extracted_settings'])}개")
print(f"   • Events: {len(callback_data['extracted_events'])}개")
print(f"   • Relationships: {len(callback_data['relationship_graph'].get('relationships', []))}개")

## 전체 파이프라인 요약

In [ ]:
print("\n" + "="*70)
print("📊 FastAPI Multi-Agent Pipeline 완료 요약")
print("="*70)

print(f"\n✅ Phase 1 - Extraction Layer:")
print(f"   • Characters: {len(state['extracted_characters'])}명")
print(f"   • Settings: {len(state['extracted_settings'])}개")
print(f"   • Events: {len(state['extracted_events'])}개")

print(f"\n✅ Phase 2 - Analysis Layer:")
print(f"   • Relationships: {len(state['relationship_graph'].get('relationships', []))}개")
print(f"   • Conflicts: {len(state['consistency_report'].get('conflicts', []))}개")

print(f"\n✅ Phase 3 - Validation Layer:")
is_valid = state['validation_result'].get('is_valid', False)
print(f"   • Validation: {'PASSED ✅' if is_valid else 'FAILED ❌'}")

print("\n" + "="*70)
print("🎉 FastAPI 파이프라인 완료!")
print(f"→ POST {state['callback_url']}")
print("→ Spring Boot에서 PostgreSQL/Neo4j 저장 및 Image Gen 진행")
print("="*70)